# Homework 2: Information Retrieval
## Instructions
1. Students will form teams of three people each and submit a single homework for each team in the format - ID1_ID2_ID3.ipynb
2. Groups of four are not allowed and you are more than welcome to form groups of two.
2. **Do not write your names anywhere.**
3. For the code part: 
> **Write your code only in the mentioned sections. Do not change the code of other sections**. Do not use any imports unless we say so.
4. For theoretical questions, if any - write your answer in the markdown cell dedicated to this task, in **English**.


#### Deviation from the aforementioned  instructions will lead to reduced grade
---


## Clarifications
1. The same score for the homework will be given to each member of the team.  
2. The goal of this homework is to test your understanding of the concepts presented in the lectures. \
If a topic was not covered in detail during the lecture, you are asked to study it online on your own. 
Anyhow, we provide here detailed explanations for the code part and if you have problems - ask.
3. Questions can be sent to the forum, you are encouraged to ask questions but do so after you have been thinking about your question. 
4. The length of the empty gaps (where you are supposed to write your code) is a recommendation (the amount of space took us to write the solution) and writing longer code will not harm your grade. We do not expect you to use the programming tricks and hacks we used to make the code shorter.   
Having said that, we do encourage you to write good code and keep that in mind - **extreme** cases may be downgraded.  
We also encourage to use informative variable names - it is easier for us to check and for you to understand. 

For your convenience, , the code has a **DEBUG** mode that you may use in order to debug with toy data.  
It is recommended to solve the code in that mode (with efficiency in mind) and then run the code on all the data.
**Do not forget to file the HW with DEBUG == False**.

`Download the data` from [HERE](https://www.kaggle.com/gyani95/380000-lyrics-from-metrolyrics) and put it in the same directory your script is.

Since it is the first time we provide this homework please notify us if there is a bug/something is unclear, typo's exc..

5. We use Python 3.7 for programming.
6. Make sure you have all the packages and functions used in the import section. Most of it is native to Anaconda Python distribution.

### Have fun !

# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline
from tqdm import tqdm
from typing import List,Dict
from IPython.display import Image
from IPython.core.display import HTML 
from pathlib import Path

In [2]:
from nltk.tokenize import word_tokenize 
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download("stopwords")
nltk.download("punkt")
from string import punctuation, ascii_lowercase
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\omerw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\omerw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Debug
""" you can change this cell """

In [3]:
DEBUG = False
"""
Recommended to start with a small number to get a feeling for the preprocessing with prints (N_ROWS_FOR_DEBUG = 2)
later increase this number for 5*10**3 in order to see that the code runs at reasonable speed. 
When setting Debug == False, our code implements bow.fit() in 15-20 minutes according to the tqdm progress bar. Your solution is not supposed to be much further than that.
"""
N_ROWS_FOR_DEBUG = 5*10**3 

# Config

In [4]:
INPUT_FILE_PATH = Path("lyrics.csv")
BOW_PATH = Path("bow.csv")
N_ROWS = N_ROWS_FOR_DEBUG if DEBUG else None
CHUNCK_SIZE = 5 if DEBUG else 5*10**3
tqdm_n_iterations = N_ROWS//CHUNCK_SIZE +1 if DEBUG else 363*10**3//CHUNCK_SIZE + 1
COLS = [5]

## 1.1 Bag of words /TfIdf model
### Implement the following methods:

* `preprocess_sentence`: 
    * Lower case the word
    * Ignores it if it's in the stopwords list
    * Removes characters which are not in the allowed symbols
    * Stems it and appends it to the output sentence
    * Discards words with length <= 1
    
    
* `update_counts_and_probabilities`: 

    * Update self.unigram count (the amount of time each word is in the text)
    * Update self.bigram count (two consecutive word occurances)
    * Update self.trigram count (three consecutive word occurances)
    * Update inverted index: a dictionary with words as keys and the values is a dictionary - {'DocID' : word_count}   
    
* `compute_word_document_frequency`:

   * For each word count the number of docs it appears in. For example , for the word 'apple' -
$$\sum_{i \in docs} I(apple \in doc_i), I := Indicator function$$


* `update_inverted_index_with_tf_idf_and_compute_document_norm`:

    * Update the inverted index (which currently hold word counts) with tf idf weighing. We will compute tf by dividing with the number of words in each document. 
    * As we want to calculate the document norm, incrementally update the document norm. pay attention that later we apply sqrt to it to finish the process.

#### The result of this code is a bag of words model that already counts for TF-IDF weighing

In [5]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
allowed_symbols = set(l for l in ascii_lowercase)

In [6]:
def preprocess_sentence(sentence : str) -> List[str]:
    output_sentence = []
    for word in word_tokenize(sentence):
        ### YOUR CODE HERE
        word = str(np.char.lower(word))
        tmp_word = ''
        for letter in word:
            if letter in allowed_symbols:
                tmp_word += letter
        word = tmp_word
        if (word in stop_words):
            next
        else:
            word = stemmer.stem(word)
            if len(word) > 1:
                output_sentence.append(word)


                          
        ### END YOUR CODE
    return output_sentence
    

def get_data_chuncks() -> List[str]:
    for i ,chunck in enumerate(pd.read_csv(INPUT_FILE_PATH, usecols = COLS, chunksize = CHUNCK_SIZE, nrows = N_ROWS)):
        chunck = chunck.values.tolist()
        yield [chunck[i][0] for i in range(len(chunck))] 

class TfIdf:
    def __init__(self):
        self.unigram_count =  Counter()
        self.bigram_count = Counter()
        self.trigram_count = Counter()
        self.document_term_frequency = Counter()
        self.word_document_frequency = {}
        self.inverted_index = {}
        self.doc_norms = {}
        self.n_docs = -1
        self.sentence_preprocesser = preprocess_sentence
        self.bow_path = BOW_PATH

    def update_counts_and_probabilities(self, sentence :List[str],document_id:int) -> None:
        sentence_len = len(sentence)
        self.document_term_frequency[document_id] = sentence_len
        for i,word in enumerate(sentence):
            ### YOUR CODE HERE
            self.unigram_count.update([word])
            if i+1 < len(sentence):
                self.bigram_count.update([word +' '+ sentence[i+1]])
            if i + 2 < len(sentence):
                self.trigram_count.update([word +' '+ sentence[i + 1]  +' '+ sentence[i + 2]])
            if word not in self.inverted_index:
                self.inverted_index[word] = {}
            if document_id not in self.inverted_index[word]:
                self.inverted_index[word][document_id] = 1
            else:
                self.inverted_index[word][document_id] += 1
            

                        
            ### END YOUR CODE
        
        
    def fit(self) -> None:
        for chunck in tqdm(get_data_chuncks(), total = tqdm_n_iterations):
            for sentence in chunck: #sentence is a song (string)
                self.n_docs += 1 
                if not isinstance(sentence, str):
                    continue
                sentence = self.sentence_preprocesser(sentence)
                if sentence:
                    self.update_counts_and_probabilities(sentence,self.n_docs)
        self.save_bow() # bow is 'bag of words'
        self.compute_word_document_frequency()
        self.update_inverted_index_with_tf_idf_and_compute_document_norm()
             
    def compute_word_document_frequency(self):
        for word in self.inverted_index.keys():
            ### YOUR CODE HERE
             self.word_document_frequency[word] = len(self.inverted_index[word])

            ### END YOUR CODE
            
    def update_inverted_index_with_tf_idf_and_compute_document_norm(self):
        ### YOUR CODE HERE
        for word in self.inverted_index:
            for document in self.inverted_index[word]:
                self.inverted_index[word][document] /= float(self.word_document_frequency[word])
                #norm
                if document not in self.doc_norms:
                    self.doc_norms[document] = self.inverted_index[word][document]**2
                else:
                    self.doc_norms[document] += self.inverted_index[word][document]**2
        

                
        ### END YOUR CODE
        for doc in self.doc_norms.keys():
            self.doc_norms[doc] = np.sqrt(self.doc_norms[doc]) 
            
    def save_bow(self):
        pd.DataFrame([self.inverted_index]).T.to_csv(self.bow_path)
                
tf_idf = TfIdf()
tf_idf.fit()

100%|██████████████████████████████████████████████████████████████████████████████████| 73/73 [45:51<00:00, 37.69s/it]


## 1.11 Bag of words model:

1. What is the computational complexity of this model, as a function of the number of docs in the corpus?
2. How can we make this code better in terms running time (parallelization or other topics you find)? 

## 1.2 DocumentRetriever
Not this retriever &#8595;


![dsafdsafsdafdsf](https://cdn3-www.dogtime.com/assets/uploads/2019/10/golden-cocker-retriever-mixed-dog-breed-pictures-cover-1.jpg)

### Implement the following methods:

`reduce_query_to_counts`: given a list of words returns a counter object with words as keys and counts as values.

`rank`: given a query and relevant documents calculate the similarity (cosine or inner product simialrity) between each document and the query.   
Make sure to transform the query word counts to tf idf as well. 

`sort_and_retrieve_k_best`: returns the top k documents.

In [7]:
class DocumentRetriever:
    def __init__(self, tf_idf):
        self.sentence_preprocesser = preprocess_sentence  
        self.vocab = set(tf_idf.unigram_count.keys())
        self.n_docs = tf_idf.n_docs
        self.inverted_index = tf_idf.inverted_index
        self.word_document_frequency = tf_idf.word_document_frequency
        self.doc_norms = tf_idf.doc_norms
        
    def rank(self,query : Dict[str,int],documents: Dict[str,Counter],metric: str ) -> Dict[str, float]:
        result = {} # key: DocID , value : float , simmilarity to query
        query_len = np.sum(np.array(list(query.values())))
        ### YOUR CODE HERE
        for word in query:
            #print(word)
            query[word] /= query_len
            #print('query[word] ',query[word])
            for doc in documents[word]:
                if doc not in result:
                    result[doc] = query[word] * documents[word].get(doc)
                else:
                    result[doc] += query[word] * documents[word].get(doc)
               
         ### END YOUR CODE
        if metric == 'cosine':
            ### YOUR CODE HERE
            for v in result:
                #print (result[doc], self.doc_norms[doc])
                result[v] /= (self.doc_norms[v])    #if we will divide in the query norm all the values the rank will be the same so no need

        #print({key: result[key] for key in sorted(result, key=result.get, reverse=True)[:5]})

        return result
         ### END YOUR CODE
    
    def sort_and_retrieve_k_best(self, scores: Dict[str, float],k :int):
        ### YOUR CODE HERE 
        return sorted(scores, key=scores.get, reverse=True)[:k]

        ### END YOUR CODE

    def reduce_query_to_counts(self, query: List) -> Counter:
        ### YOUR CODE HERE
        return Counter(query)

        ### END YOUR CODE
        
        
    def get_top_k_documents(self,query : str, metric: str , k = 5) -> List[str]:
        query = self.sentence_preprocesser(query)
        query = [word for word in query if word in self.vocab] # filter nan 
        query_bow = self.reduce_query_to_counts(query)
        relavant_documents = {word : self.inverted_index.get(word) for word in query}
        ducuments_with_similarity = self.rank(query_bow,relavant_documents, metric)
        return self.sort_and_retrieve_k_best(ducuments_with_similarity,k)
        
dr = DocumentRetriever(tf_idf)

In [166]:
from IPython.display import HTML
query = "Better stop dreaming of the quiet life, 'cause it's the one we'll never know And quit running for that runaway bus 'cause those rosy days are few And stop apologizing for the things you've never done 'Cause time is short and life is cruel but it's up to us to change This town called malice"
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/KT6ZtUbVw1M?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

c:\users\omerw\appdata\local\programs\python\python37\lib\site-packages\IPython\core\display.py:694: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [167]:
cosine_top_k = dr.get_top_k_documents(query, 'cosine')
print(cosine_top_k)
inner_product_top_k = dr.get_top_k_documents(query, 'inner_product')
print(inner_product_top_k)

[68983, 189048, 233333, 109203, 100158]
[18869, 123673, 211934, 129254, 225923]


In [170]:
for index, song in enumerate(pd.read_csv(INPUT_FILE_PATH,usecols = [5]).iloc[cosine_top_k]['lyrics']):
    sep = "#"*50
    print(F"{sep}\nsong #{index} \n{song} \n{sep}")

##################################################
song #0 
Why
Why don't you like me?
Why
Why don't you like me?
Why
Why don't you like me?
Why
Why don't you like me? 
##################################################
##################################################
song #1 
Feel me
Feel me
Feel me
... 
##################################################
##################################################
song #2 
One, two, one, two, three, four
I feel like I do
(Like I do)
I feel like I do
I feel like I do
(Like I do)
All the years, I see
(I see)
I feel like I do
(Like I do)
I feel like I do
(Like I do)
Let me know how you feel
Let me know how you feel
I feel like I do
(Like I do)
I feel like I do
(Like I do)
All the years, I see
(I see) 
##################################################
##################################################
song #3 
You should feel what I feel
You should take what I take
You should feel what I feel
You should take what I take
You should feel what I fe

# 1.3 term statistics:
Use "tf_idf" object that we created earlier and answer the following questions:

1. How many unique words we have?
2. How many potential word bigrams we have? How many actual word bigrams we have? How do you explain this difference?
3. What is the storage size of the input file "lyrics.csv"? What is the output file (bow.csv) size? how do you explain this difference?  

In [11]:
# 1. 
### YOUR SOLUTION HERE
len(tf_idf.unigram_count)


### END YOUR SOLUTION

"""
### Your verbal solution here
We have 388243  unique words

### End your verbal solution here
"""

# 2.
### YOUR SOLUTION HERE

# if every pair of words where unique se we had
sum(tf_idf.bigram_count.values())
# because not all the pairs are unique, we have
len(tf_idf.bigram_count)
#which is
len(tf_idf.bigram_count)
### END YOUR SOLUTION
#percent from the maximum value
"""
### Your verbal solution here

if every pair of words where unique se we had 32,595,363 pairs.
because not all the pairs are unique, we have 6,312,785 pairs,
which is 19.3 percent from the maximum value
the different is due to the  pairs of words in our corpus, which repeat themselves. 

### End your verbal solution here
"""

# 3.
### YOUR SOLUTION HERE
print('Lyrics size: ',INPUT_FILE_PATH.stat().st_size / 1000.0, ' KB')
print('bow size: ',BOW_PATH.stat().st_size / 1000.0, ' KB')
### END YOUR SOLUTION

"""
### Your verbal solution here
Lyrics size:  324,632.382  KB
bow size:  193,319.838  KB

the lyrics file contains all the words of all the documents. the bow file contains just the unique words in each document (even that after removing some 
illegal signs and stemming all the words) . 
The BOW method allows us to save our data in a numeric way instead of long strings
the scores of a words in the documents don't cost us so much as the size of the original documents.


### End your verbal solution here
"""

Lyrics size:  324632.382  KB
bow size:  193319.838  KB


"\n### Your verbal solution here\nLyrics size:  324,632.382  KB\nbow size:  193,319.838  KB\n\nthe lyrics file contains all the words of all the documents. the bow file contains just the unique words in each document (even that after removing some \nillegal signs and stemming all the words) . \nThe BOW method allows us to save our data in a numeric way instead of long strings\nthe scores of a words in the documents don't cost us so much as the size of the original documents.\n\n\n### End your verbal solution here\n"

## 1.4 NgramSpellingCorrector
Now we will implement a Ngarm (character Ngrams) spelling corrector. That is, we have an out of vocabulary word (v) and we want to retrieve the most similar words (in our vocabulary) to this word.
we will model the similarity of two words by-

$$sim(v,w) := prior \cdot likelihood = p(w) \cdot P(v|w) $$ 
$$P(v|w) := JaccardIndex =  \frac{|X \cap Y|}{|X \cup Y|}$$

Where v is an out of vocabulary word (typo or spelling mistake), w is in a vocabulary word, X is the ngram set of v and Y is the ngram set of w.
For example, if n == 3, the set of ngrams for word "banana" is set("ban","ana","nan","ana") = {"ban","ana","nan"}

In order to do it efficently, we will first construct an index from the possible Ngrams we have seen in our corpus to the words that those Ngrams appear in, in order prevent comparing v to all of the words in our corpus.
Then, we will implement a function that computes this similarity.

* Make sure you compute the JaccardIndex efficently!

In [12]:
def get_bigrams(word):
    for ngram in nltk.ngrams(word, 2):
        yield "".join(list(ngram))
    
def get_trigrams(word):
    for ngram in nltk.ngrams(word, 3):
        yield "".join(list(ngram))
        
        
"""
for example - get_bigrams is a generator, which is an object we can loop on:
for ngram in get_bigrams(word):
    DO SOMETHING
"""

'\nfor example - get_bigrams is a generator, which is an object we can loop on:\nfor ngram in get_bigrams(word):\n    DO SOMETHING\n'

In [13]:
class NgramSpellingCorrector:
    def __init__(self, unigram_counts: Counter, get_n_gram: callable):
        self.unigram_counts = unigram_counts
        self.ngram_index = {}
        self.get_n_grams = get_n_gram
    
    def build_index(self) -> None:
        ### YOUR CODE HERE
          for word in self.unigram_counts:
                self.ngram_index[word] = self.get_n_grams(word)

        
        
        
        ### END YOUR CODE
        
    def get_top_k_words(self,word:str,k=5) -> List[str]:
        ### YOUR CODE HERE

        all_unigrams_count = sum(self.unigram_counts.values())
        word_ngram = list(self.get_n_grams(word))
        #intersect
        #for word_ng in word_ngram:
        for ngram in self.ngram_index:
            #print('ngram',ngram)
            #print('word_ngram',word_ngram)
            p_w = self.unigram_counts[ngram]/all_unigrams_count
            self.ngram_index[ngram] = p_w * round(len((set(word_ngram).intersection(list(self.ngram_index[ngram])))) / len((set(word_ngram).union((list(self.ngram_index[ngram]))))), 2)
        return sorted(self.ngram_index, key=self.ngram_index.get, reverse=True)[:k]

        
                
        
        ### END YOUR CODE


class BigramSpellingCorrector(NgramSpellingCorrector):
    def __init__(self, unigram_counts: Counter):
        super().__init__(unigram_counts, get_bigrams)
        
        
class TrigramSpellingCorrector(NgramSpellingCorrector):
    def __init__(self, unigram_counts: Counter):
        super().__init__(unigram_counts, get_trigrams)
        

In [14]:
tf_idf.unigram_count['money']

38345

In [222]:
out_of_vocab_word = 'luve'
bigram_spelling_corrector = BigramSpellingCorrector(tf_idf.unigram_count)
bigram_spelling_corrector.build_index()
bigram_spelling_corrector.get_top_k_words(out_of_vocab_word)

['nt', 'want', 'dont', 'went', 'moment']

In [94]:
trigram_spelling_corrector = TrigramSpellingCorrector(tf_idf.unigram_count)
trigram_spelling_corrector.build_index()
trigram_spelling_corrector.get_top_k_words(out_of_vocab_word)

['look', 'lookin', 'took', 'cool', 'hook']

## 1.5 Language model
Calculate the log likelihood of a sentence. Once with a bigram markovian langauge model, and once with a trigram model.
for example - the likelihood of the senetence "spiderman spiderman does whatever a spider can" for the bigram model is: 
$$p(spiderman)\cdot p(spiderman|spiderman) \cdot  p(does|spiderman) \cdot p(whatever|does) \cdot  p(a|whatever) \cdot  p(spider|a) \cdot p(can|spider)$$

And for the trigram model:
$$p(spiderman,spiderman)\cdot p(does|spiderman,spiderman) \cdot  p(whatever|spiderman,does) \cdot p(a|does,whatever) \cdot  p(spider|whatever,a) \cdot  p(can|a, spider)$$

Since we do not want a zero probability sentence use Laplace smoothing, as you have seen in the lecture, or here https://en.wikipedia.org/wiki/Additive_smoothing

In [221]:
import math
# for the probability smoothing
NUMERATOR_SMOOTHING = 1 # alpha in https://en.wikipedia.org/wiki/Additive_smoothing
DENOMINATOR_SMOOTHING = 10**4 # d in https://en.wikipedia.org/wiki/Additive_smoothing
def sentence_log_probabilty(unigrams : Counter, bigrams  : Counter,trigrams : Counter, sentence: str):
    bigram_log_likelilhood, trigram_log_likelilhood = 0, 0
    words_in_sentence = sentence.split()
    n_words = len(words_in_sentence)
    all_unigrams_count = sum(unigrams.values())
    all_bigrams_count = sum(bigrams.values())
    all_trigrams_count = sum(trigrams.values())
    words_dict = []
    p_w_bi = []
    p_w_tri = []
    alpha= 1
    d = 10**4
    
    for index, word in  enumerate(words_in_sentence):
        ### YOUR CODE HERE
        words_dict.append(word)
        
        if index == 0:
            p_w_bi.append((unigrams[words_dict[0]] + alpha) /(all_unigrams_count + d))
        if index == 1:
            temp_bi = (bigrams[words_dict[(index-1)] + ' ' + words_dict[index]] + alpha)/(all_bigrams_count + d) # to delete
            p_w_tri.append(temp_bi)
            p_w_bi.append((temp_bi/(unigrams[words_dict[index-1]] + alpha)/(all_unigrams_count + d)))
        if (index > 1):
            temp_bi = (bigrams[words_dict[(index-1)] + ' ' + words_dict[index]] + alpha)/(all_bigrams_count + d)
            p_w_bi.append((temp_bi/(unigrams[words_dict[index-1]] + alpha)/(all_unigrams_count + d)))
            temp_tri = (trigrams[words_dict[(index-2)] + ' ' + words_dict[(index-1)] + ' ' + words_dict[index]] + alpha)/(all_trigrams_count + d)
            p_w_tri.append((temp_tri /(bigrams[words_dict[index-2] + ' ' + words_dict[index-1]] + alpha)/(all_bigrams_count + d)))

            

    bigram_log_likelilhood = sum([np.log(x) for x in p_w_bi])
    trigram_log_likelilhood = sum([np.log(x) for x in p_w_tri])
    
    print(F"Bigram log likelihood is {bigram_log_likelilhood}")
    print(F"Trigram log likelihood is {trigram_log_likelilhood}")
    return bigram_log_likelilhood, trigram_log_likelilhood

    
sentence = "green red red "
sentence_log_probabilty(tf_idf.unigram_count, tf_idf.bigram_count, tf_idf.trigram_count, sentence)

Bigram log likelihood is -85.36258410538174
Trigram log likelihood is -51.891917608100655


(-85.36258410538174, -51.891917608100655)

## 1.51 Language model: B
For each model what is the next word prediciton for the sentnence "i am"?

In [220]:
### YOUR CODE HERE
def perdicted_word(unigrams : Counter, bigrams  : Counter,trigrams : Counter, sentence: str):
    res_bi = {}
    res_tri = {}
    words = sentence.split()
    words_len = len(words)
    if words_len < 2:
        return ('sentence is to short for trigrams model')
    for i in temp:
        bi_gram = words[-1] + ' ' + i
        tri_gram = words[-2] + ' ' + words[-1] + ' ' + i
        
        if bigrams[bi_gram] > 0:
            sentence_temp= sentence + ' ' + i
            bigram_log,trigram_log = sentence_log_probabilty(unigrams, bigrams,trigrams, sentence_temp)
            res_bi[i] = bigram_log
            
            if trigrams[tri_gram] > 0:
                res_tri[i] = trigram_log
    
    if len(res_bi) == 0:
        print('\n no word match for bigram method')
    
    else:
        print('\n bigram model prediction is {}'.format(sorted(res_bi, key=res_bi.get, reverse=True)[0:5]))
        
    if len(res_tri) == 0:
        print('\n no word match for trigram method')
    
    else:
        print('\n trigram model prediction is {}'.format(sorted(res_tri, key=res_tri.get, reverse=True)[0:5]))
    

### END YOUR CODE
perdicted_word(tf_idf.unigram_count, tf_idf.bigram_count, tf_idf.trigram_count, 'feel like')


Bigram log likelihood is -81.86358185235466
Trigram log likelihood is -47.65171403741526
Bigram log likelihood is -81.90408883663449
Trigram log likelihood is -47.89287609423214
Bigram log likelihood is -81.27041864271297
Trigram log likelihood is -47.155277151101366
Bigram log likelihood is -82.57645604870268
Trigram log likelihood is -47.09618823473136
Bigram log likelihood is -84.95651645386502
Trigram log likelihood is -50.513914918344724
Bigram log likelihood is -84.63806272274648
Trigram log likelihood is -49.82076773778478
Bigram log likelihood is -83.17172565395305
Trigram log likelihood is -47.805864717242514
Bigram log likelihood is -85.20783088214593
Trigram log likelihood is -49.12762055722484
Bigram log likelihood is -84.569743478769
Trigram log likelihood is -51.90020927946462
Bigram log likelihood is -83.36955139728298
Trigram log likelihood is -48.855686841741196
Bigram log likelihood is -82.7827121507369
Trigram log likelihood is -46.20647714066192
Bigram log likelihoo